In [134]:
import pandas as pd
import numpy as np

Importing the Movies and rating dataset...

In [135]:
movie = pd.read_excel('movies.xlsx')

In [136]:
rating = pd.read_excel('ratings.xlsx')

Individual tables for movie and their ratings...

In [137]:
movie.head(10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [138]:
rating.head(10)

,userId,movieId,rating,timestamp
0,1,1,4,964982703
1,1,3,4,964981247
2,1,6,4,964982224
3,1,47,5,964983815
4,1,50,5,964982931
5,1,70,3,964982400
6,1,101,5,964980868
7,1,110,4,964982176
8,1,151,5,964984041
9,1,157,5,964984100


Combining movie and rating table

In [139]:
mergedmovierating = pd.merge(rating, movie, on = 'movieId')
mergedmovierating.head(10)

,userId,movieId,rating,timestamp,title,genres
0,1,1,4,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4,964982224,Heat (1995),Action|Crime|Thriller


In [140]:
columnsvalue = ['timestamp', 'genres']
mergedmovierating = mergedmovierating.drop(columnsvalue, axis=1)
mergedmovierating.head(10)

,userId,movieId,rating,title
0,1,1,4,Toy Story (1995)
1,1,3,4,Grumpier Old Men (1995)
2,1,6,4,Heat (1995)


In [141]:
mergedmovierating = mergedmovierating.dropna(axis = 0, subset = ['title'])

movie_ratingCount = (mergedmovierating.
     groupby(by = ['title'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'Total Rating'})
     [['title', 'Total Rating']]
    )
movie_ratingCount.head(10)

,title,Total Rating
0,Grumpier Old Men (1995),1
1,Heat (1995),1
2,Toy Story (1995),1


In [142]:
rating_with_totalRatingCount = mergedmovierating.merge(movie_ratingCount, left_on = 'title', right_on = 'title', how = 'left')
rating_with_totalRatingCount.head(10)

,userId,movieId,rating,title,Total Rating
0,1,1,4,Toy Story (1995),1
1,1,3,4,Grumpier Old Men (1995),1
2,1,6,4,Heat (1995),1


In [143]:
user_rating = rating_with_totalRatingCount.drop_duplicates(['userId','title'])
user_rating.head(10)

,userId,movieId,rating,title,Total Rating
0,1,1,4,Toy Story (1995),1
1,1,3,4,Grumpier Old Men (1995),1
2,1,6,4,Heat (1995),1


Matrix Factorization (Creating a matrix and fill 0 values)

In [144]:
movie_user_rating_pivot = user_rating.pivot(index = 'userId', columns = 'title', values = 'rating').fillna(0)
movie_user_rating_pivot.head(10)

title,Grumpier Old Men (1995),Heat (1995),Toy Story (1995)
userId,,,
1,4,4,4


In [145]:
X = movie_user_rating_pivot.values.T
X = X*(X.T)
X.shape


(3, 3)

In [146]:
import sklearn
from sklearn.decomposition import TruncatedSVD

SVD = TruncatedSVD(2)
matrix = SVD.fit_transform(X)
matrix.shape

(3, 2)

In [147]:
import warnings
warnings.filterwarnings("ignore",category =RuntimeWarning)
corr = np.corrcoef(matrix)
corr.shape


(3, 3)

Output....

In [148]:
movie_title = movie_user_rating_pivot.columns
movie_title_list = list(movie_title)
coffey_hands = movie_title_list.index("Grumpier Old Men (1995)")

In [149]:
corr_coffey_hands  = corr[coffey_hands]
list(movie_title[(corr_coffey_hands >= 0.9)])

['Grumpier Old Men (1995)', 'Heat (1995)', 'Toy Story (1995)']